In this post, I'd like to look at three stochastic dynamical systems, each a little more complex than the next.
We'll mainly be concerned with differential equations of the form

$$\dot z = f(z) + \sigma\dot\xi$$

where the state space that $z$ lives in is $\mathbb{R}^n$, $f : \mathbb{R}^n \to \mathbb{R}^n$ is the deterministic part of the dynamics, $\dot\xi$ is random *white noise*, and $\sigma$ is the noise amplitude.
In general, $\sigma$ could be a matrix that depends on the solution $z$ itself, but for the following we'll only consider cases where $\sigma$ is a scalar constant.

If you haven't seen stochastic differential equations before, you might be wondering at this point what I mean by "white noise".
White noise is characterized as a Gaussian stochastic process with zero mean and covariance function

$$\langle \dot\xi(s)\dot\xi(t)\rangle = \delta(t - s)$$

where $\delta$ is the Dirac delta function and $\langle\cdot\rangle$ denotes statistical expectation.
Like the delta function, white noise is a bit of a convenient fiction that we tell ourselves in order to get on with our lives -- there is no well-defined stochastic process with these properties.
Defining [stochastic integration](https://en.wikipedia.org/wiki/It%C3%B4_calculus) from first principles is... pretty rowdy and I don't want to go there.
If you want to see the gory details, you can refer to the books by [Gardiner](https://link.springer.com/book/9783540707127) or [Øksendal](https://doi.org/10.1007/978-3-642-14394-6).

For our purposes, it's enough to know that the solutions of this stochastic differential equation can emerge through a limit of discrete-time processes

$$z_{n + 1} = z_n + \delta t\cdot F(z_n, z_{n + 1}) + \sqrt{\delta t}\cdot\sigma\varepsilon_n,$$

where $\varepsilon_n$ are uncorrelated mean-zero normal random variables with variance 1, and $F$ is some function of the current and past state that makes for a consistent discretization of the deterministic part of the ODE.
(I'm leaving room here for both explicit and implicit time discretizations.)
There are no gross issues of existence or regularity here because $\{\varepsilon_n\}$ has variance 1, unlike the weird delta function business with the continuous-time SDE.
The first process that we'll look at can be solved in closed form, but the next two have no analytical solution and so numerical methods are our only recourse.

### The Ornstein-Uhlenbeck process

The simplest stochastic dynamical systems we can consider are the linear ones:

$$\dot z = -\gamma z + \sigma\dot\xi,$$

where $\gamma$ is a symmetric positive-definite matrix.
Ornstein originally used this system as a model for the velocity of a particle under the combined influence of friction and random collision with neighboring particles.
With the notation above, $\gamma$ is the friction coefficient and $\sigma$ is the collision intensity.

Much like for deterministic ODE, we can use the matrix exponential and the usual [variation of parameters](https://en.wikipedia.org/wiki/Variation_of_parameters) trick to write down an explicit solution of this SDE:

$$z(t) = e^{-t\gamma}z(0) + \sigma\int_0^te^{-(t - s)\gamma}\dot\xi(s)\, ds.$$

Since the noise $\dot\xi$ is a Gaussian process and any finite sum of normal random variables is also normal, the solution $z$ is also a Gaussian process.
If we can compute the mean and covariance function, then we know almost everything we need about the process.

First, taking expectations can be interchanged with time integrals by Fubini's theorem.
So if we take the expected value of the solution we'll be left with just the exponentially-decaying influence of the initial condition:

$$\mu(t) = \langle z(t)\rangle = e^{-t\gamma}\langle z(0)\rangle.$$

I'm leaving the expectations on the initial condition here because it too might be non-deterministic.
The correlation function is much more interesting:

$$\begin{align}
\text{corr}(t, t + \tau) & \equiv \sigma^{-2}\langle (z(t) - \mu(t))(z(t + \tau) - \mu(t + \tau))^*\rangle \\
& = \int_0^{t + \tau}\int_0^t e^{-(t - s_1)\gamma}e^{-(t + \tau - s_2)\gamma}\langle\dot\xi(s_1)\dot\xi(s_2)^*\rangle ds_1\,ds_2\ldots \\
\end{align}$$

The correlation between the noise terms is $\delta(s_1 - s_2)$, but we have to remember that the integration limits on $s_2$ extend to a range $t + \tau$ not included in the integration limits on $s_1$.
Since the $\delta$ distribution is zero there, we can exclude this part of the integral:

$$\begin{align}
\ldots & = \int_0^t\int_0^te^{-(t - s_1)\gamma}e^{-(t + \tau - s_2)\gamma}\delta(s_1 - s_2)\, ds_1\,ds_2 \\
& = \int_0^te^{-(t - s)\gamma)}e^{-(t + \tau - s)\gamma}ds \\
& = e^{-\tau\gamma}.
\end{align}$$

We can also extend this to the negative $\tau$ case as $\text{corr}(t, t + \tau) = \exp\left(-|\tau|\gamma\right)$.
There are two important things to observe here.
First, the correlation function decays exponentially as the separation $\tau$ increases.
Second, the correlation only depends on the difference $\tau$ between the two time points and not the value of the base time point $t$.
This means that the process is *stationary*.

So the OU process is Gaussian and stationary, and, like all solutions of SDEs forced by white noise, it has the Markov property.
The final fact (which I find to be just amazing) is that *any other* stochastic process with these three properties is identical to the OU process up to scaling.

That's enough theory for now, let's write a procedure to generate sample paths.
Assuming that the decay rate $\gamma$ is positive, we'll want some kind of implicit scheme to integrate the SDE in order to guarantee stability independent of the stepsize.
The implicit Euler scheme will be good enough for our purposes.
There is an [exact procedure](https://doi.org/10.1103/PhysRevE.54.2084) for simulating the OU process, but it doesn't generalize to the more complex SDEs that we'll consider in the following.

In [ ]:
import numpy as np

def ornstein_uhlenbeck(z, γ, σ, dt, N, rng):
    zs = np.zeros(N + 1)
    zs[0] = z
    ε = rng.standard_normal(N)
    
    for n in range(N):
        zs[n + 1] = (zs[n] + np.sqrt(dt) * σ * ε[n]) / (1 + γ * dt)
        
    return zs

In [ ]:
from numpy import random

rng = random.default_rng(seed=1729)
dt = 0.005
N = 10000
zs = ornstein_uhlenbeck(z=0.0, γ=0.1, σ=0.01, dt=dt, N=N, rng=rng)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ts = np.linspace(0, dt * N, N + 1)
ax.plot(ts, zs);

In [ ]:
fig, ax = plt.subplots()

num_samples = 5
for sample in range(num_samples):
    zs = ornstein_uhlenbeck(z=0.0, γ=0.1, σ=0.01, dt=dt, N=N, rng=rng)
    ax.plot(ts, zs, linewidth=0.75)

### The Schlogl reaction

The Ornstein-Uhlenbeck process plays a similar role in stochastic dynamical systems to the coupled harmonic oscillator for classical physics and mechanics -- a linear, exactly solvable model that can be used for understanding a host of other nonlinear models near to equilibrium.
We can view the Ornstein-Uhlenbeck process as a stochastically-perturbed gradient flow for the potential

$$\phi(z) = \frac{\gamma}{2}|z|^2,$$

In other words, the deterministic part of the system dynamics $\dot z = -\nabla\phi + \sigma\dot\xi$ move $z$ towards extrema of the potential $\phi$.
Here we'll look at a more complex problem: stochastic motion in a double-well potential

$$\phi(z) = \frac{\gamma}{4\zeta^2}(\zeta^2 - |z|^2)^2,$$

which has minima at $z = \pm \zeta$ and a single maximum at $z = 0$.
When the noise level is 0, the system will remain near the minima of the potential forever, but the addition of noise means that the system can migrate between these equilibria on some sufficiently long time scale.

We'll use sympy to calculate all the derivatives, not because they're particularly difficult to do by hand, but because we'll want to consider yet more complex problems where they are.

In [ ]:
from sympy import symbols, lambdify, diff

z = symbols("z")
ζ, γ = symbols("ζ γ", positive=True)

ϕ = γ / (4 * ζ ** 2) * (z ** 2 - ζ ** 2) ** 2
dϕ = lambdify((z, ζ, γ), diff(ϕ, z))
d2ϕ = lambdify((z, ζ, γ), diff(ϕ, z, 2))

When we integrated the Ornstein-Uhlenbeck process numerically, we used the implicit Euler scheme, which is easy for scalar linear equations.
Using the implicit Euler scheme for the Schlogl reaction would entail a lot more work.
Instead, we'll use a *linearly* implicit Euler scheme; rather than solve that nonlinear equation to its full accuracy, we'll only do a single step of Newton's method.
We'll need that $\delta t < \alpha\gamma^{-1}$ for some $\alpha < 1$ in order to guarantee that we get a well-defined update in every step.

In [ ]:
def schlogl_process(z, ζ, γ, σ, dt, N, rng):
    zs = np.zeros(N + 1)
    zs[0] = z
    ε = rng.standard_normal(N)
    
    for n in range(N):
        f = dt * dϕ(zs[n], ζ, γ) - np.sqrt(dt) * σ * ε[n]
        df = 1 + dt * d2ϕ(zs[n], ζ, γ)
        zs[n + 1] = zs[n] - f / df
        
    return zs

Looking at a single realization of the Schlogl reaction, we can see that on long enough time scales the system oscillates back and forth between the two stable equilibria.
While we can't calculate the residence times directly, we can use some of the insight we gained about first-passage times for the OU process.

In [ ]:
dt = 0.025
N = 50000
rng = random.default_rng(seed=1729)
zs = schlogl_process(z=-1.0, ζ=1.0, γ=0.1, σ=0.25, dt=dt, N=N, rng=rng)

In [ ]:
fig, ax = plt.subplots()
ts = np.linspace(0.0, N * dt, N + 1)
ax.plot(ts, zs);

### Cascading tipping points

The final process we'll consider is from the 2018 paper [Cascading transitions in the climate system](https://doi.org/10.5194/esd-9-1243-2018) by Dekker, von der Hydt, and Dijkstra.
The idea behind the models they present is to use a Schlogl-type model where the coefficients themselves are determined by Schlogl-type models with different characteristic time scales.